# 인사이트

 본문을 찾아내는 방법. `newspaper3k` 라이브러리의 방법은 일단 소거하고, 남은 것 중에서도 링크가 달려 있는 것 없는 것. 공백이 많으면? 등등 자기 방법을 고민해 보자.

# 오늘의 느낀 점

1. url 파라미터를 넘기자.
2. link 가져오는 방법
    - 나는 href로 찾아 오는데,
    - 강사님은 selector 이용해서 바로 찾는다. **selector로 찾는 게 더 편하다!!!!!**
3. pagination 이용하는 다른 방법.
    - 나는 마지막 페이지만 찾아서 반복문 돌렸는데,
    - 강사님은 아예 페이지네이션의 링크를 가져 온다.
4. selector 이용할 때 주의
    - `>` : 태그의 직계 자식만을 의미. `find_all`에서 `recursive=False` 옵션을 준 것과 동일.
    - ` ` : 공백은 하위 태그 전체를 의미.
5. HTML 페이지 가져올 때 광고 등 빼낼 수 있는 방법은?
    - HTML 포맷 보고 빼주거나,
    - HTML 페이지 다 가져와서 바뀌는 부분만 남기고 소거하거나.

# 자주 만나는 오류

![error](./images/web_09.png)


- SSL 에러 : HTTP, HTTPS 때문에 생김.
    - `주의 요함` 크롬 브라우저에 떠 있는 경우/`안전하지 않다`는 경우.
    - 현재 url은 `https://`로 되어 있다. 그런데 인증받지 못해서 문제가 생길 수 있음.
    
- request 요청 보낼 때 verify 옵션을 넣어 준다. : `requests.get(url, verify=False)`



In [101]:
# module import
import requests
from bs4 import BeautifulSoup
import re
from pprint import pprint

(0601 연습문제 이어서)
* 내가 푼 방법은 `find`했기 때문에 뉴스 토픽만 나옴.

In [2]:
# 20200602
# 강사님 정답

# module import
import requests
from bs4 import BeautifulSoup

In [19]:
# url 요청
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%ED%95%9C%EA%B5%AD%EC%9D%80%ED%96%89&oquery=%EB%AC%B4%EC%97%AD%EC%A0%84%EC%9F%81&tqi=UWTPrdprvmssskwbSNwssssstPG-019933"
req = requests.get(url)
req # 접속 성공 확인

<Response [200]>

In [11]:
soup = BeautifulSoup(req.text) # soup 객체 생성

# find/find_all로 찾기
ols = soup.find_all('ol', class_='lst_realtime_srch')
len(ols) # 확인용 : 첫 번째는 뉴스, 두 번째는 연예/스포츠.

# 뉴스 및 연예, 스포츠를 zip을 써서 unpack한다.
for news_li, enter_li in zip(*ols): # 확인용
    print(news_li, enter_li)
    break

<li><a class="bx bx_item" href="?where=nexearch&amp;sm=tab_htk.nws&amp;ie=utf8&amp;query=5%EC%9B%94+%EC%86%8C%EB%B9%84%EC%9E%90%EB%AC%BC%EA%B0%80+0.3%25%E2%86%93" onclick="return goOtherCR(this, 'a=htk.nwslist&amp;r=1&amp;i=&amp;u='+urlencode(urlexpand(this.href)));"><span class="keyword"><em class="num">1</em><span class="tit">5월 소비자물가 0.3%↓</span></span><em class="rank new"><span class="spim"></span><span>NEW</span></em></a></li> <li><a class="bx bx_item" href="?where=nexearch&amp;sm=tab_htk.ent&amp;ie=utf8&amp;query=%EB%AF%B8%EC%8A%A4%ED%84%B0%ED%8A%B8%EB%A1%AF+%EC%A0%95%EB%8F%99%EC%9B%90X%EB%82%A8%EC%8A%B9%EB%AF%BC" onclick="return goOtherCR(this, 'a=htk.entslist&amp;r=1&amp;i=&amp;u='+urlencode(urlexpand(this.href)));"><span class="keyword"><em class="num">1</em><span class="tit">미스터트롯 정동원X남승민</span></span><em class="rank new"><span class="spim"></span><span>NEW</span></em></a></li>


In [18]:
result_news_list = []
result_enter_list = []

# class가 'tit'인 내용만 불러 오면 된다.
for news_li, enter_li in zip(*ols):
    news_kwd = news_li.find('span', class_='tit').get_text()
    enter_kwd = enter_li.find('span', class_='tit').get_text()
    
    # 결과 저장
    result_news_list.append(news_kwd)
    result_enter_list.append(enter_kwd)
    
print(result_news_list)
print(result_enter_list)

['5월 소비자물가 0.3%↓', '흑인 사망 시위', '1분기 성장률', '다우 0.36%↑', '8개월만에 마이너스 물가', '대청호에 차량 추락', '3명 사망 2명 부상', '오늘 영장심사', '전 세계 확산', 'G7 초청 기꺼이 응할 것']
['미스터트롯 정동원X남승민', '해군 문화 홍보단 지원', '동상이몽2 정찬성', '백종원 백파더', '첫방 가족입니다', '장제원 아들 노엘', '이연희 오늘 결혼', '박보검 해군 군악병 지원', '가족입니다 원미경', '미스터트롯 발매']


In [41]:
# select 활용
ols = soup.select('ol.lst_realtime_srch span.tit')
# class가 lst_realtime_srch인 ol의 자식으로 있는 span인데(바로 밑 자식은 아님) class가 title.
ols # span만 한 번에 가져 옴.

[<span class="tit">5월 소비자물가 0.3%↓</span>,
 <span class="tit">흑인 사망 시위</span>,
 <span class="tit">1분기 성장률</span>,
 <span class="tit">다우 0.36%↑</span>,
 <span class="tit">8개월만에 마이너스 물가</span>,
 <span class="tit">대청호에 차량 추락</span>,
 <span class="tit">3명 사망 2명 부상</span>,
 <span class="tit">오늘 영장심사</span>,
 <span class="tit">전 세계 확산</span>,
 <span class="tit">G7 초청 기꺼이 응할 것</span>,
 <span class="tit">미스터트롯 정동원X남승민</span>,
 <span class="tit">해군 문화 홍보단 지원</span>,
 <span class="tit">동상이몽2 정찬성</span>,
 <span class="tit">백종원 백파더</span>,
 <span class="tit">첫방 가족입니다</span>,
 <span class="tit">장제원 아들 노엘</span>,
 <span class="tit">이연희 오늘 결혼</span>,
 <span class="tit">박보검 해군 군악병 지원</span>,
 <span class="tit">가족입니다 원미경</span>,
 <span class="tit">미스터트롯 발매</span>]

In [42]:
ols = [ol.get_text() for ol in ols]

news_list = []
enter_list = []

for news_k, enter_k in zip(ols[:10], ols[10:]):
    news_list.append(news_k)
    enter_list.append(enter_k)

print(news_list)
print(enter_list)

['5월 소비자물가 0.3%↓', '흑인 사망 시위', '1분기 성장률', '다우 0.36%↑', '8개월만에 마이너스 물가', '대청호에 차량 추락', '3명 사망 2명 부상', '오늘 영장심사', '전 세계 확산', 'G7 초청 기꺼이 응할 것']
['미스터트롯 정동원X남승민', '해군 문화 홍보단 지원', '동상이몽2 정찬성', '백종원 백파더', '첫방 가족입니다', '장제원 아들 노엘', '이연희 오늘 결혼', '박보검 해군 군악병 지원', '가족입니다 원미경', '미스터트롯 발매']


### HTML 구조

```
<!DOCTYPE html>
<html lang> ... : html 태그
<head> ... </head> : 바디를 잘 보여주기 위한 메타데이터, description.
<body> ... </body>: 구성요소. 눈에 보이는 것. html 태그들이 포함되어 있음.
```

(사진)

<연습문제 4> 

(이어 풀기)

* 질문 : 블로그 스크레이핑이 안 돼요 ㅠㅠ

#### 원래 안 되는 거예요!! 다른 방법을...

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from urllib.parse import urlencode

In [26]:
query = '기아타이거즈'
QUERY = urlencode({'query':query}, encoding='utf-8')

In [27]:
# 기본 블로그 url
blog_url = f"https://search.naver.com/search.naver?{QUERY}&sm=tab_pge&srchby=all&st=sim&where=post"
print(blog_url)

https://search.naver.com/search.naver?query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&srchby=all&st=sim&where=post


In [28]:
# 블로그 글 5페이지까지 돌면서 link 가져오기
links = []

for i in range(5):
    url = f"{blog_url}&start={i*10 + 1}"
    req = requests.get(url)
    print(req) # 체크용
    
    soup = BeautifulSoup(req.text)
    anchors = soup.find_all('a', {'class' : 'sh_blog_title'})
    print(len(anchors))
    
    for anchor in anchors:
        links.append(anchor['href'])

print(len(links))

<Response [200]>
10
<Response [200]>
10
<Response [200]>
10
<Response [200]>
10
<Response [200]>
10
50


In [46]:
anchors

[<a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/pickmon_04?Redirect=Log&amp;logNo=221971211151" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=41&amp;i=90000003_0000000000000033AE83E38F&amp;u='+urlencode(this.href))" target="_blank" title="[KBO] 05월20일 기아타이거즈 롯데자이언츠 : 이 투수가 선발로 나왔을 때 아직까지 득점지원이 0점이죠?">[KBO] 05월20일 <strong class="hl">기아타이거즈</strong> 롯데자이언츠 : 이 투수가 선발로...</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/formation_9776?Redirect=Log&amp;logNo=221971894600" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=42&amp;i=90000003_0000000000000033AE8E5148&amp;u='+urlencode(this.href))" target="_blank" title="2020년5월21일 롯데자이언츠 vs 기아타이거즈 / 국내야구 집중분석">2020년5월21일 롯데자이언츠 vs <strong class="hl">기아타이거즈</strong> / 국내야구...</a>,
 <a class="sh_blog_title _sp_each_url _sp_each_title" href="https://blog.naver.com/80cjm?Redirect=Log&amp;logNo=221960677427" onclick="return goOtherCR(this, 'a=blg*i.tit&amp;r=43&amp;i=90

In [30]:
for link in links:
    print(link)

https://blog.naver.com/jyong2288?Redirect=Log&logNo=221972724113
https://blog.naver.com/psbear?Redirect=Log&logNo=221981936696
https://blog.naver.com/tmdgh2040?Redirect=Log&logNo=221981265398
https://toxicnaomi.blog.me/221947006629
https://blog.naver.com/pkijj04?Redirect=Log&logNo=221975138084
https://blog.naver.com/imaplus?Redirect=Log&logNo=221951270588
https://blog.naver.com/nabibelle?Redirect=Log&logNo=221977027811
https://blog.naver.com/grappler39?Redirect=Log&logNo=221946915356
https://blog.naver.com/ekschf?Redirect=Log&logNo=221986637239
https://blog.naver.com/pickmon_04?Redirect=Log&logNo=221981732445
https://blog.naver.com/hanjinlee21?Redirect=Log&logNo=221956697661
https://blog.naver.com/gkxodnjsdjflsdl?Redirect=Log&logNo=221968761493
https://blog.naver.com/tou2me?Redirect=Log&logNo=221951806448
https://qorskawn5.blog.me/221960514369
https://blog.naver.com/vervexerox?Redirect=Log&logNo=221982941615
https://blog.naver.com/niceccc?Redirect=Log&logNo=221986558513
https://blog.na

In [ ]:
# 각 블로그에서 제목과 글 작성자, 글 내용 가져오기

for i in range(len(links)):
    req = requests.get(links[i])
    
    if i % 5 == 0: # for check
        print(i, req)
    
    soup = BeautifulSoup(req.text)
    
    title = soup.find()

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')

In [45]:
# 확인용

test_url = "https://blog.naver.com/jyong2288?Redirect=Log&logNo=221972724113"

driver = webdriver.Chrome('./chromedriver.exe')
driver.get(test_url)

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "se-component-content"))
    )
finally:
    driver.quit()


# driver.get(test_url)
# driver.implicitly_wait(5)
html = driver.page_source
print(html)

TimeoutException: Message: 


<연습문제 3.5, 4 강사님 풀이>

* **params**를 넘기자~

In [ ]:
# 강사님 정답 1) : 뉴스 링크 가져오기 

base_url = "https://search.naver.com/search.naver?where=post&sm=tab_jum&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88"

result_list = []

# url crawling
for i in range(1, 50, 10):
    url = base_url.format(i)
    
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    anchor_title_tag = soup.select('ul.type01>li a._sp_each_title')
    for anchor_tag in anchor_title_tag:
        result_list.append({
            'title' : anchor_tag.text,
            'url' : anchor_tag.get('href')
        })
    

In [103]:
# 강사님 정답 2) : 뉴스 링크 가져오기

base_url = "https://search.naver.com/search.naver"

query = input("검색어를 입력해주세요. ")
page_limit = int(input("몇 페이지까지 가져올까요? "))

params = {'where': 'news', 
         'query': query,
         'start': 0}

result_list = []

for i in range(page_limit):
    params['start'] = i*10 + 1# 검색 결과 페이지 바꿔주기 위함.
    resp = requests.get(base_url, params=params)
    
    soup = BeautifulSoup(resp.text)
    anchor_title_tag = soup.select('ul.type01>li a._sp_each_title')
    for anchor_tag in anchor_title_tag:
        result_list.append({
            'title' : anchor_tag.text,
            'url' : anchor_tag.get('href')
        })

pprint(result_list)

검색어를 입력해주세요. 양현종
몇 페이지까지 가져올까요? 3
[{'title': "[SC핫플레이어]'4승, 0.51' 구창모, '우상' 양현종 넘었다",
  'url': 'http://sports.chosun.com/news/ntype.htm?id=202006020100017000000824&servicedate=20200602'},
 {'title': '대투수 양현종 등판 전날 온다고 부담스러워 하더니 外[SS 주간방담]',
  'url': 'http://www.sportsseoul.com/news/read/922371?ref=naver'},
 {'title': '멈춰버린 스카우팅리포트…양현종은 MLB에 진출할 수 있을까',
  'url': 'http://yna.kr/AKR20200601178700007?did=1195m'},
 {'title': '구창모, 양현종 류현진 시대 마감 선언 [성일만 야구선임기자의 핀치히터]',
  'url': 'http://www.fnnews.com/news/202006011446092652'},
 {'title': "[B스토리] 류현진·양현종도 비켜라! 올해는 구창모가 '왕'이다",
  'url': 'http://star.mt.co.kr/stview.php?no=2020060210085398644'},
 {'title': "NC 구창모의 '역대급' 초구 스트라이크 비율 '76.4%'",
  'url': 'http://yna.kr/AKR20200602032400007?did=1195m'},
 {'title': '"흥미로운 매치업" 윌리엄스의 소형준 경계&양현종 칭찬[MD토크]',
  'url': 'http://www.mydaily.co.kr/new_yk/html/read.php?newsid=202005281739639857&ext=na'},
 {'title': "[IS 수원]KIA 양현종, KT전 시즌 최다 피안타+실점...'고전'",
  'url': 'http://isplus.live.joins.com/news/article/

* pagination 접근에서는 start가 없다.
* pagination을 나타내는 태그를 찾는다
    - `div.paging`
    - 현재 페이지: strong으로 감싸져 있음.
* 페이지 번호마다 링크가 있다.

In [107]:
# 강사님 정답 3 : pagination 접근

base_url = "https://search.naver.com/search.naver"

query = input("검색어를 입력해주세요. ")

params = {'where': 'news', 
         'query': query}

# pagination을 통해 
resp = requests.get(base_url, params=params)
soup = BeautifulSoup(resp.text)

paging_wrap_tag = soup.find('div', class_='paging')
active_page = paging_wrap_tag.find('strong').text
other_pages = paging_wrap_tag.find_all('a')

# dictionary comprehension 이용해 dict에 저장.
page_list = {
    a_tag.text.strip():a_tag.get('href') for a_tag in other_pages
}

# url을 만든다.
for i in 

검색어를 입력해주세요. 기아타이거즈


{'2': '?&where=news&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=19&start=11',
 '3': '?&where=news&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=19&start=21',
 '4': '?&where=news&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=19&start=31',
 '5': '?&where=news&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:r,p:all,a:all&mynews=0&cluster_rank=19&start=41',
 '6': '?&where=news&query=%EA%B8%B0%EC%95%84%ED%83%80%EC%9D%B4%EA%B1%B0%EC%A6%88&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=0&ds=&de=&docid=&nso=so:

<연습문제 4>
 네이버 뉴스 상세페이지 조회하기

* 연예 뉴스 주의

![entertainment](./images/web_10.png)

In [137]:
base_url = "https://search.naver.com/search.naver?"

In [138]:
query = "한국"
where = "news"
QUERY = urlencode({'query':query}, encoding='utf-8')
WHERE = urlencode({'where':where}, encoding='utf-8')
print(QUERY)
print(WHERE)

query=%ED%95%9C%EA%B5%AD
where=news


In [139]:
search_url = f"{base_url}&{WHERE}&{QUERY}"
print(search_url)

https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD


In [140]:
# 지정한 페이지까지 돌면서 뉴스 결과 가져오기

limit = 5

for i in range(limit):
    url = f"{search_url}&start={10*i + 1}"
    print(url)

https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD&start=1
https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD&start=11
https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD&start=21
https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD&start=31
https://search.naver.com/search.naver?&where=news&query=%ED%95%9C%EA%B5%AD&start=41


In [141]:
# 뉴스 링크 가져오기
links = []

for i in range(limit):
    prev = len(links)
    
    url = f"{search_url}&start={10*i + 1}"
    req = requests.get(url)
    print(req.status_code)
    
    soup = BeautifulSoup(req.text)
    temp = soup.find_all('a', {'href' : re.compile('.+news.naver.com/.+')})
    for t in temp:
        link = t['href']
        if link not in links and 'aid' in link: # aid 없는 뉴스면 article ID 없다고 판단해서 제외.
            links.append(link)

    print(f"{i+1}page : Found {len(links) - prev} links.")
        
print(len(links))

200
1page : Found 16 links.
200
2page : Found 19 links.
200
3page : Found 11 links.
200
4page : Found 18 links.
200
5page : Found 14 links.
78


In [142]:
results = []

for link in links:
    req = requests.get(link)
    print(link, req.status_code)
    soup = BeautifulSoup(req.text)
    
    try:        
        title = soup.find('h3', {'id' : 'articleTitle'}).get_text(strip=True)
        press = soup.find('div', {'class' : 'press_logo'}).find('img')['alt']
        article = soup.find('div', {'id' : 'articleBodyContents'}).get_text(strip=True)
        
    except: # 연예뉴스인 경우
        title = soup.find('h2', {'class' : 'end_tit'}).get_text(strip=True)
        press = soup.find('div', {'class' : 'press_logo'}).find('img')['alt']
        article = soup.find('div', {'id' : 'articeBody'}).get_text(strip=True)        
    
    print("done")
    
    results.append({
        'url':link,
        'title':title,
        'press':press,
        'article':article
                   })

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=025&aid=0003005762 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=586&aid=0000014110 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=011&aid=0003747132 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=014&aid=0004435439 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=016&aid=0001681124 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=023&aid=0003535539 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=025&aid=0003005378 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=020&aid=0003289414 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=001&aid=0011646355 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0009892204 200
done
https://news.naver.com/main/read.nhn?mode=LSD&mid=

In [148]:
print(results[:2])
print(len(results))

[{'url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=025&aid=0003005762', 'title': '한국 등 G7회의 초청 못마땅한 日 "정식 멤버는 아닐 것"', 'press': '중앙일보', 'article': '외무성 간부 "특별 초청국으로 참가할 듯"아시아 유일 선진국 日 존재감 후퇴 우려연일 부정적 반응, 스가 "G7 역할 중요"도널드 트럼프 미국 대통령이 한국과 호주·인도·러시아 등을 주요 7개국(G7)정상회의에 초청한 것과 관련해 일본내에서 부정적인 반응이 이어지고 있다.문재인 대통령이 지난해 6월 28일 일본 오사카에서 열린 G20 정상회의 환영식에서 아베 신조 일본 총리(왼쪽)와 8초간 악수한 뒤 이동하고 있다. [청와대사진기자단]트럼프 대통령은 1일 문재인 대통령과의 통화에서 G7 체제를 “낡은 체제로서 현재의 국제정세를 반영하지 못한다”고 표현하며 이를 G11 또는 G12 체제로 확대하겠다는 뜻을 밝혔다.2일 일본 산케이 신문에 따르면 일본 외무성 관계자는 "정상회의의 멤버를 갑자기 4개국 늘리는 건 무리"라고 말했다.G7을 G11이나 G12로 늘리려면 기존 G7 참가국들의 동의가 필요하기 때문에 올해 의장국인 미국이 마음대로 정하기 어렵다는 것이다.산케이 신문은 "트럼프 대통령이 (미국내)실무진과도 사전에 조정하지 않고 발언한 것으로 보여,일본 외무성도 트럼프 대통령의 진의 파악에 어려움을 겪고 있다”고 했다.이와관련, 외무성 간부는 산케이에 “우선은 (정식 멤버가 아닌 초청국을 뜻하는)아웃리치의 형태로 초청하지 않겠는가”라고 했다.정식 멤버가 아니라 의장국이 추가로 초청하는 국가나 국제기관의 자격으로 회의에 참석할 가능성이 크다는 뜻이다.일본은 트럼프 대통령의 구상대로 G7이 확대돼 한국 등이 추가로 참가하는 데 대해 못마땅하다는 입장을 공공연하게 드러내고 있다.전날 요미우리 신문엔 “(한국과 호주 등이 참가하면)아시아 유일의 G7 정상회의 참가

## 이후 클래스화할 때 반영

In [149]:
# 아래 방식으로 하면 네이버 뉴스 홈페이지가 아니라서 크롤링할 때 플랫폼별로 다르게 해야 한다!

In [58]:
re.compile(r'sp_news')

re.compile(r'sp_news', re.UNICODE)

In [ ]:
# 스포츠 뉴스 링크 가져오기 : html을 가져오는 방식으로 하자.

links = []

for i in range(limit):
    prev = len(links)
    url = f"{search_url}&start={10*i + 1}"
    req = requests.get(url)
    print(req.status_code)
    soup = BeautifulSoup(req.text)
    temp = soup.find_all('li', {'id': re.compile('sp_nws\d+')})
    
    for t in temp:
        for a in t.find_all('a'):
            link = a['href']
            if link.startswith('http') and link not in links:
                links.append(link)
    
    print(f"{i+1}page : Found {len(links) - prev} links.")
        
print(links)

* dd 아래 description 태그

In [125]:
# 강사님 풀이

# 네이버 뉴스 리스트 저장
base_url = "https://search.naver.com/search.naver"
params = {
    'where':'news',
    'query':'무역전쟁'
}

# 네이버 뉴스 리스트 request & save
for i in range(int(input('몇 페이지까지 스크레이핑할까요?'))):
    resp = requests.get(base_url, params=params, verify=False)
    soup = BeautifulSoup(resp.text)
    news_list_items = soup.select('ul.type01 > li')
    
    for news_item in news_list_items:
        anchor_title = news_item.find('a', class_ = '_sp_each_title')
        desc = news_item.find_all('dd')[1]
        title = anchor_title.text
        url = anchor_title.get('href')
        
        # send request to news url
        news_detail_resp = requests.get(url, verify=False)
        content = news_detail_resp.text
        
        result_list.append({
            'title' : title,
            'desc' : desc,
            'url' : url,
            'content' : content
        })
        
        break
        

몇 페이지까지 스크레이핑할까요?3


C:\Users\student\.conda\envs\multi_sir950123\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\student\.conda\envs\multi_sir950123\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.fnnews.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\student\.conda\envs\multi_sir950123\lib\site-packages\urllib3\connectionpool.py:986: InsecureRequestWarning: Unverified HTTPS request is being made to host 'search.naver.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings